In [17]:
#
# Example - Using uvatradier to fetch quote data for a given symbol
#

import os, dotenv; import pandas as pd;
from uvatradier import Tradier, Quotes

# Create .env file within current working directory.
# Add the following to your .env file. Specify the correct account number and authorization token for your quote.
# 	tradier_acct = <ACCOUNT_NUMBER>
#	tradier_token = <AUTH_TOKEN>

dotenv.load_dotenv()

ACCOUNT_NUMBER = os.getenv('tradier_acct')
AUTH_TOKEN = os.getenv('tradier_token')

# Initializing new Quotes object
quotes = Quotes(ACCOUNT_NUMBER, AUTH_TOKEN);

data = quotes.get_historical_quotes('KO', start_date='2022-10-01', end_date='2023-10-01');

print(data);

# data['close_change'] = data['close'].pct_change()
# data['volume_change'] = data['volume'].pct_change()\
close_change = data['close'].pct_change();
volume_change = data['volume'].pct_change();

           date   open    high     low  close    volume
0    2022-10-03  56.36  56.850  56.060  56.65  16277620
1    2022-10-04  56.95  57.810  56.460  56.78  18861431
2    2022-10-05  56.50  56.650  55.510  56.24  14324851
3    2022-10-06  56.00  56.100  54.870  55.03  17192843
4    2022-10-07  54.99  55.090  54.250  54.51  11479799
..          ...    ...     ...     ...    ...       ...
245  2023-09-25  57.40  57.485  56.740  57.00  12358824
246  2023-09-26  56.73  56.895  56.400  56.53  11194700
247  2023-09-27  56.42  56.530  55.680  55.95  13766301
248  2023-09-28  56.23  56.320  55.700  55.81  12484769
249  2023-09-29  56.14  56.270  55.615  55.98  12271561

[250 rows x 6 columns]


In [18]:
print(close_change)

0           NaN
1      0.002295
2     -0.009510
3     -0.021515
4     -0.009449
         ...   
245   -0.010417
246   -0.008246
247   -0.010260
248   -0.002502
249    0.003046
Name: close, Length: 250, dtype: float64


In [19]:
volume_change

0           NaN
1      0.158734
2     -0.240522
3      0.200211
4     -0.332292
         ...   
245   -0.156129
246   -0.094194
247    0.229716
248   -0.093092
249   -0.017077
Name: volume, Length: 250, dtype: float64

In [23]:
# Given the close_change Series (as per your example)
# close_change = pd.Series([None, 0.002295, -0.009510, -0.021515, -0.009449], name='close')  # truncated for brevity

# Replace NaN values with the mean of the non-NaN values
close_change_filled = close_change.fillna(close_change.mean())
volume_change_filled = volume_change.fillna(volume_change.mean())

# print(close_change_filled)
# print(volume_change_filled)


logit_data = pd.DataFrame({'close':close_change_filled, 'volume':volume_change_filled});
print(logit_data)


        close    volume
0   -0.000009  0.057599
1    0.002295  0.158734
2   -0.009510 -0.240522
3   -0.021515  0.200211
4   -0.009449 -0.332292
..        ...       ...
245 -0.010417 -0.156129
246 -0.008246 -0.094194
247 -0.010260  0.229716
248 -0.002502 -0.093092
249  0.003046 -0.017077

[250 rows x 2 columns]


In [31]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# # Assuming these are your Series (based on provided data)
# close = pd.Series([
#     -0.000009, 0.002295, -0.009510, -0.021515, -0.009449, # truncated for brevity
#     -0.010417, -0.008246, -0.010260, -0.002502, 0.003046
# ])

# volume = pd.Series([
#     0.057599, 0.158734, -0.240522, 0.200211, -0.332292, # truncated for brevity
#     -0.156129, -0.094194, 0.229716, -0.093092, -0.017077
# ])

# Convert close to binary. Assuming a positive change is 1 and non-positive is 0.
logit_data['close_binary'] = (logit_data['close'] > 0).astype(int)

# Convert the Series to a DataFrame for training
logit_data = pd.DataFrame({
    'volume': logit_data['volume'],
    'close_binary': close_binary
})

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(logit_data[['volume']], logit_data['close_binary'], test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression()

# Fit the model
model.fit(X_train, y_train)

# Get the score of the model
score = model.score(X_test, y_test)

print(f"Model Accuracy: {score*100:.2f}%")


ImportError: dlopen(/opt/homebrew/Caskroom/miniconda/base/lib/python3.10/site-packages/scipy/spatial/_distance_wrap.cpython-310-darwin.so, 0x0002): symbol not found in flat namespace (_npy_copysign)